# Gray Scale Model

### Important Variables:

In this task we will be implementing the SIR model with the following system parameters:

*   $a_{i}(b)$: total amount of assets hold by user i by the end of block b.
*   $d_{i}(b)$: total amount of black assets hold by node i by the end of block b.
*   $x_{i}(b)$: = $\frac{d_{i}(b)}{a_{i}(b)} $ black assets fraction by the end of block b.
*   $c_{i}(b)$: amount of clean asset owned by node i by the end of block b
*   $m_{i}(b)$: the amount mined by user i by the end of block b
*   $W(b)$: = {$w_{ij}$}$_{i,j=1,...,N}$ is the adjacency matrix of the transaction network at block b. wij is equal to the amount of bitcoin moved from user i to user j at block b.
*   $N = Nusers = |Nusers|$ total number of users in the network.

Below is a summary of the dynamics of the SIR model:

In [2]:
import blocksci

import sys, os, os.path, socket
import numpy as np
import zarr
import time
import pandas as pd
from collections import defaultdict
from decimal import Decimal
import datetime
import math

from util import SYMBOLS, DIR_BCHAIN, DIR_PARSED, SimpleChrono, darknet


def format_e(n):
    a = '%E' % n
    return a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]

# Address Mapper Class

In [3]:
# this class creates a map from addresses to the set {0 -> number of cluster}, in other words it helps to use arrays to store the information. 
class AddressMapper(): # This creates the map that clusters the addresses to actual users. Treat it a a blackbox for now.
    def __init__(self, chain):
        self.chain = chain

        self.__address_types = [blocksci.address_type.nonstandard, blocksci.address_type.pubkey,
                                blocksci.address_type.pubkeyhash, blocksci.address_type.multisig_pubkey,
                                blocksci.address_type.scripthash, blocksci.address_type.multisig,
                                blocksci.address_type.nulldata, blocksci.address_type.witness_pubkeyhash,
                                blocksci.address_type.witness_scripthash, blocksci.address_type.witness_unknown]

        self.__counter_addresses = { _:self.chain.address_count(_) for _ in self.__address_types }

        self.__offsets = {}
        offset = 0
        for _ in self.__address_types:
            self.__offsets[_] = offset
            offset += self.__counter_addresses[_]


        self.total_addresses = offset
        print(f"[INFO] #addresses: {self.total_addresses}")
#        print(self.__counter_addresses)


    def map_clusters(self,cm):
#        address_vector = {_: np.zeros(self.__counter_addresses[_], dtype=np.int64) for _ in self.__address_types }
        cluster_vector = {_: np.zeros(self.__counter_addresses[_], dtype=np.int64) for _ in self.__address_types }

        self.cluster = np.zeros(self.total_addresses, dtype=np.int64)
        offset = 0
        for _at in cluster_vector.keys():
            clusters = cluster_vector[_at]
            print(f"{_at}     -  {len(clusters)}")
#            addrs = address_vector[_at]
            for _i, _add in enumerate(chain.addresses(_at)):
#                addrs[_i] = _add.address_num
                clusters[_i] = cm.cluster_with_address(_add).index
                #max_addr_num = max(max_addr_num, addrs[_i])
#        pickle.dump(cluster_vector, open("cluster_dict.pickle","wb"))

        offset = 0
        for _ in cluster_vector.keys():
            v = cluster_vector[_]
            self.cluster[offset:offset + len(v)] = v
            offset += len(v)



    def dump_clusters(self, output_folder):
        if not os.path.exists(output_folder):
            os.mkdir(output_folder)
        zarr.save(f"{output_folder}/address_cluster_map.zarr", self.cluster)


#    def dump_offsets(self, output_folder):
#        if not os.path.exists(output_folder):
#            os.mkdir(output_folder)
#        pickle.dump(self.__offsets, open(f"{output_folder}/offsets.pickle", "wb") )

#    def load_offsets(self, output_folder):
#        if not os.path.exists(output_folder):
#            os.mkdir(output_folder)
#        self.__offsets = pickle.load( open(f"{output_folder}/offsets.pickle", "rb") )

    def load_clusters(self, input_folder):
        print(f"{input_folder}address_cluster_map.zarr")
        self.cluster = zarr.load(f"{input_folder}address_cluster_map.zarr")



    def __getitem__(self,addr):
        return self.__offsets[addr.raw_type]+ addr.address_num-1

def catch(address, am):
    try:
        return am[address] 
    except:
        return np.nan

## Total number of addresses and clusters

In [4]:
chain = blocksci.Blockchain(f"{DIR_PARSED}/bitcoin_old.cfg") # load the blockchain
am = AddressMapper(chain)
am.load_clusters(f"{DIR_PARSED}/bitcoin/heur_1_data/") #this data should already be on the server.

no_clusters = max( am.cluster ) + 1 # print the number of clusters
print(f"[INFO] #clusters: {no_clusters}")

[INFO] #addresses: 4022303272
/export/consensus-2/blockchain_parsed/bitcoin/heur_1_data/address_cluster_map.zarr
[INFO] #clusters: 591692716


# Counting number of True and False values in cluster_is_black_ground_truth 

In [16]:
array_zarr = zarr.load('uniform_black/heur_1_data/cluster_is_black_ground_truth.zarr') #load zarr file into array
df = pd.DataFrame (array_zarr) ## convert your array into a dataframe
print(len(df.index))
#filepath = 'zarr_to_excel.csv'
#df.to_csv(filepath, index=False)
countTrue = np.where(array_zarr==True)
countFalse = np.where(array_zarr==False)
element_count_False = countFalse[0].size
element_count_True = countTrue[0].size
print(len(array_zarr))
print(element_count_False)
print(element_count_True)
print(array_zarr)
for i in countTrue:
    print(i)

591692716
591692716
591692581
135
[False False False ... False False False]
[ 79800777  82285475  86826333  87290351  88316586  89576173  90613521
  92044383  93820427  93821719  93822603  94285328  94377857  94494584
  94507944  94616297  94620783  94624325  94668900  94797389  94887197
  94890855  94892403  94894707  94897322  94898493  94898632  94900520
  94903701  94904451  94905527  94907956  94910188  94912551  94913697
  94913957  94914365  94915941  94918924  94918931  94921217  94922115
  94922151  94922197  94922679  94926589  94927243  94928208  94934381
  94934648  94937864  94939326  94941929  94943306  94943818  94944952
  94944968  94946270  94946799  94947040  94947412  94947783  94948795
  94949293  94949444  94950027  94950285  94950484  94951587  94952593
  94954100  94959563  94960057  94961888  94962603  94963307  94966001
  94966614  94967639  94968329  94969379  94969678  94969686  94971061
  94971748  94972606  94972700  94974446  94974455  94974547  94974967
 

In [15]:
df = pd.read_csv(f"{DIR_PARSED}/bitcoin_darknet/ground_truth_id.csv")

print(len(df.index))
print(df.shape[0])
# print(df)

28027931
28027931


In [6]:
df = pd.read_csv(f"uniform_black/heur_1_data/ground_truth_clust_id.csv")

print(len(df.index))
print(df.shape[0])
# print(df)

1822562
1822562
